<a href="https://colab.research.google.com/github/Diljitsingh14/AIML/blob/main/SMOL/smol_SFT_ex.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -U bitsandbytes datasets trl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 MB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 293.4/293.4 kB 15.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 13.8 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [3]:
# Huggingface login using colab secrets [make sure you will add secret into notebook]
from huggingface_hub import login
from google.colab import userdata


login(token=userdata.get('hf'))

In [27]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from datasets import load_dataset
import torch
from trl import SFTTrainer,SFTConfig,setup_chat_format

model_name = "HuggingFaceTB/SmolLM2-135M"
device = ("cuda" if torch.cuda.is_available() else "cpu")

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(pretrained_model_name_or_path = model_name)

model , tokenizer = setup_chat_format(model=model,tokenizer=tokenizer)

finetune_name = "SmolLM2-FT-MyDataset"
finetune_tags = ['smol-course',"module_1"]

In [28]:
promt = "Write a haiku about programming"

message = [{"role":"user","content":promt}]
formatted_prompt = tokenizer.apply_chat_template(message,tokenize=False)

inputs = tokenizer(formatted_prompt,return_tensors="pt")
outputs = model.generate(
                        **inputs,
                         max_new_tokens=100,
                          temperature=0.9,  # Increase randomness
                          top_p=0.85,      # Use nucleus sampling
                          repetition_penalty=1.2,  # Penalize repeated sequences
                          pad_token_id=tokenizer.eos_token_id

                         )
print("Before Training")
print(tokenizer.decode(outputs[0],skip_special_tokens=True))

Before Training
user
Write a haiku about programming
What is the best way to write an essay?The 10 Most Important Things You Need To Know About The Internet Of Things (IoT)

25.3K reads
This article was written by Shawna Kaur, a student at the University of California Irvine and has been edited for clarity since it first appeared on the internet in January 2017. It's not meant as a replacement or substitute for any other information that you may have read before but


In [13]:
from datasets import load_dataset
ds = load_dataset(path="HuggingFaceTB/smoltalk",name="everyday-conversations")

In [14]:
# check if the dataset in the write format as chat format need ie "role" and "content" format.

ds['train'][0]['messages']

[{'content': 'Hi there', 'role': 'user'},
 {'content': 'Hello! How can I help you today?', 'role': 'assistant'},
 {'content': "I'm looking for a beach resort for my next vacation. Can you recommend some popular ones?",
  'role': 'user'},
 {'content': "Some popular beach resorts include Maui in Hawaii, the Maldives, and the Bahamas. They're known for their beautiful beaches and crystal-clear waters.",
  'role': 'assistant'},
 {'content': 'That sounds great. Are there any resorts in the Caribbean that are good for families?',
  'role': 'user'},
 {'content': 'Yes, the Turks and Caicos Islands and Barbados are excellent choices for family-friendly resorts in the Caribbean. They offer a range of activities and amenities suitable for all ages.',
  'role': 'assistant'},
 {'content': "Okay, I'll look into those. Thanks for the recommendations!",
  'role': 'user'},
 {'content': "You're welcome. I hope you find the perfect resort for your vacation.",
  'role': 'assistant'}]

In [15]:
def process_dataset(sample):
    sample['text'] =  tokenizer.apply_chat_template(sample['messages'], tokenize=False)
    return sample

In [16]:
ds = ds.map(process_dataset)

Map:   0%|          | 0/2260 [00:00<?, ? examples/s]

Map:   0%|          | 0/119 [00:00<?, ? examples/s]

In [17]:
sf_config = SFTConfig(
    output_dir = "./sft_output",
    max_steps = 1000,
    per_device_train_batch_size=4,
    learning_rate=.5e-5,
    logging_steps=10,
    save_steps=100,
    evaluation_strategy="steps",
    eval_steps=50,
    # gradient_accumulation_steps=4,
    hub_model_id=finetune_name
)

trainer = SFTTrainer(
    model=model,
    args=sf_config,
    train_dataset=ds['train'],
    eval_dataset=ds['test'],
    # formatting_func=process_dataset,
    tokenizer=tokenizer,
)


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-17-e122948f7d68>:14: FutureWarning: `tokenizer` is deprecated and removed starting from version 0.16.0 for `SFTTrainer.__init__`. Use `processing_class` instead.
  trainer = SFTTrainer(


Map:   0%|          | 0/2260 [00:00<?, ? examples/s]

Map:   0%|          | 0/119 [00:00<?, ? examples/s]

In [18]:
trainer.train()
trainer.save_model(f"./{finetune_name}")

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Step,Training Loss,Validation Loss
50,1.435800,1.478068
100,1.282300,1.296784
150,1.208600,1.247198
200,1.185900,1.215045
250,1.163500,1.191987
300,1.136800,1.175992
350,1.128900,1.162315
400,1.106800,1.152883
450,1.118300,1.144504
500,1.168300,1.137895


In [26]:
# Let's test the base model before training
prompt = "Write a haiku about programming"

# Format with template
messages = [{"role": "user", "content": prompt}]
formatted_prompt = tokenizer.apply_chat_template(messages, tokenize=False)

# Generate response
inputs = tokenizer(formatted_prompt, return_tensors="pt").to(device)

In [19]:
trained_model = AutoModelForCausalLM.from_pretrained(f"./{finetune_name}")
trained_model.to(device)

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(49152, 576, padding_idx=2)
    (layers): ModuleList(
      (0-29): 30 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear(in_features=576, out_features=576, bias=False)
          (k_proj): Linear(in_features=576, out_features=192, bias=False)
          (v_proj): Linear(in_features=576, out_features=192, bias=False)
          (o_proj): Linear(in_features=576, out_features=576, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=576, out_features=1536, bias=False)
          (up_proj): Linear(in_features=576, out_features=1536, bias=False)
          (down_proj): Linear(in_features=1536, out_features=576, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((576,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((576,), eps=1e-05)
      )
    )
    (norm

In [25]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

# Load the fine-tuned model and tokenizer
device = "cuda" if torch.cuda.is_available() else "cpu"

trained_model = AutoModelForCausalLM.from_pretrained(f"./{finetune_name}").to(device)
tokenizer = AutoTokenizer.from_pretrained(f"./{finetune_name}")

# Define the input prompt
prompt = "Write a haiku about programming"

# Format the prompt using the tokenizer's existing chat template
messages = [{"role": "user", "content": prompt}]
formatted_prompt = tokenizer.apply_chat_template(messages, tokenize=False)

# Tokenize the formatted prompt
inputs = tokenizer(formatted_prompt, return_tensors="pt").to(device)

# Generate response
outputs = trained_model.generate(
     **inputs,
    max_new_tokens=100,
    temperature=0.9,  # Increase randomness
    top_p=0.85,      # Use nucleus sampling
    repetition_penalty=1.2,  # Penalize repeated sequences
    pad_token_id=tokenizer.eos_token_id
)

# Decode and print the response
response = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(response)


user
Write a haiku about programming
What is the difference between an algorithm and a program? What are some examples of algorithms in everyday life, such as sorting or counting numbers. Write your own example using one of these words: "algorithm" (computer), "program", etc., to describe what you think it means for something to be considered an algorithm/a computer program. How does this relate back to our discussion on how we use computers today? Is there anything that can't be done with code without having written programs first? Why


In [22]:
messages = [{"role": "user", "content": "What are some interesting facts about space?"}]
formatted_prompt = tokenizer.apply_chat_template(messages, tokenize=False)
print("Formatted Prompt:\n", formatted_prompt)


Formatted Prompt:
 <|im_start|>user
What are some interesting facts about space?<|im_end|>



In [23]:
print("Special Tokens:", tokenizer.special_tokens_map)


Special Tokens: {'bos_token': '<|im_start|>', 'eos_token': '<|im_end|>', 'unk_token': '<|endoftext|>', 'pad_token': '<|im_end|>', 'additional_special_tokens': ['<|im_start|>', '<|im_end|>']}


In [29]:
# Define the input prompt
prompt = "What are some interesting facts about space?"

# Format the prompt using the chat template
formatted_prompt = f"<|im_start|>user\n{prompt}<|im_end|>\n<|im_start|>assistant\n"

# Tokenize the input
inputs = tokenizer(formatted_prompt, return_tensors="pt").to(device)

# Generate a response
outputs = trained_model.generate(
    **inputs,
    max_new_tokens=100,  # Number of tokens to generate
    temperature=0.9,     # Increase randomness
    top_p=0.85,          # Nucleus sampling for diversity
    repetition_penalty=1.2,  # Penalize repeated tokens
    pad_token_id=tokenizer.eos_token_id  # Use <|im_end|> as padding
)

# Decode the generated response
response = tokenizer.decode(outputs[0], skip_special_tokens=True)

# Extract only the assistant's response
response = response.split("<|im_start|>assistant\n")[-1].split("<|im_end|>")[0].strip()

print("Assistant's Response:\n", response)


Assistant's Response:
 user
What are some interesting facts about space?
assistant
How many planets in the solar system have been discovered so far, and what is their average distance from Earth?bobby
Boy: How do you know that Jupiter has 50 moons. What's its diameter?what it looks like with a telescope or binocularsand how big does it look to us on earth?how much bigger than our moonis this planet larger then we think of as beingthe size of Mars but smallerthan Mercury which was thoughtto be only one-third the
